<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

импортируем необходимые библиотеки и сохраним данные о клиентах:

In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [70]:
data=pd.read_csv('users_behavior.csv')

выведем первые 5 строк таблицы:

In [71]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


и выведем информацию о данных:

In [72]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

проверим данные на явные дубликаты:

In [73]:
data.duplicated().sum()

0

пропусков не обнаружено, данные выгружаются без ошибок, разобьем данные на выборки для машинного обучения:

## Разбейте данные на выборки

разобьем сначала исходные данные на обучающую выборку, валидационную выборку и тестовую выборку в соотношении 3:1:1. Для этого сначала разделим данные в соотношении 3:2, а затем вторую часть разделим в отношении 1:1 

In [74]:
data_train,data_validtest = train_test_split(data, test_size=0.4, random_state=42)

data_valid,data_test=train_test_split(data_validtest, test_size=0.5, random_state=42)

проверим что все верно

In [75]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2369 to 3174
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [76]:
data_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1198 to 1510
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [77]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1545 to 283
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


размер выборок поделился в отношении 1928:643:643, то есть примерно в отношении 3:1:1. Все верно

In [78]:
# код ревьюера
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(1928, 5)
(643, 5)
(643, 5)


## Исследуйте модели

перед тем как обучить модель выделим обучающий набор данных:

выделим признаки:

In [79]:
features_train = data_train.drop(['is_ultra'], axis=1)

выделим целевой признак

In [80]:
target_train = data_train['is_ultra']

и аналоигчно для валидационной и тестовой выборок:

In [81]:
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

In [82]:
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

целевой признак явялется категорийным, поэтому у нас стоит задача классификации. Обучим модель дерева принятия решений с параметром `random_state=42` и узнаем при каком значении максимальной глубины дерева `max_depth` будет максимальное значение точности `accuracy_score`

In [83]:
best_depth = 0
best_result = 0

for i in range(1,30):
    model_tree = DecisionTreeClassifier(max_depth=i,random_state=42)
    model_tree.fit(features_train,target_train)
    predictions_valid=model_tree.predict(features_valid)
    result=accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_depth = i 
        best_result = result 

print('наилучшая глубина дерева',best_depth)
print('наилучшая точность',best_result)

наилучшая глубина дерева 8
наилучшая точность 0.7962674961119751


научишая глубина дерева оказалась равна 3, а точность равна 0.788.

теперь попробуем обучить модель случайного леса и выяснить количество деревьев `n_estimators` при котором достигается максимальное значение точности `accuracy_score`. Так как случайный лес представляет собой набор из случайных деревьев точность должна быть выше

In [84]:
best_depth = 0
best_result = 0

for i in range(1, 30):
    model_forest = RandomForestClassifier(n_estimators=i,random_state=42) 
    model_forest.fit(features_train,target_train)
    predictions_valid=model_forest.predict(features_valid)
    result=accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_depth = i
        best_result = result 
    
    
print('наилучшое количество деревьев',best_depth)
print('наилучшая точность',best_result)

наилучшое количество деревьев 16
наилучшая точность 0.7978227060653188


Точность упала, самое лучшее количество деревьев - 16

Теперь посмотрим какова точность для модели логистической регрессии

In [85]:
best_depth = 0
best_result = 0

for i in range(200, 1000):
    model = LogisticRegression(random_state=42, solver='lbfgs', max_iter=i) 
    model.fit(features_train,target_train)
    predictions_valid=model.predict(features_valid)
    result=accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_depth = i
        best_result = result 
    
print('наилучшая количество деревьев',best_depth)
print('наилучшая точность',best_result)

наилучшая количество деревьев 200
наилучшая точность 0.7402799377916018


У логистической регрессии оказался самый плохой показатель точности. У случайного леса точность повыше, а у случайного дерева самая большая точность. В следующем задании будем использовать его.

## Проверьте модель на тестовой выборке

Проверим модель на тестовой выборке используя модель случайного леса

In [86]:
model = RandomForestClassifier(n_estimators=16,random_state=42) 

Обучим модель на обуучающий данных:

In [87]:
model.fit(features_train,target_train)

RandomForestClassifier(n_estimators=16, random_state=42)

и сделаем предсказание используя тестовую выборку:

In [88]:
predictions=model.predict(features_test)

## (бонус) Проверьте модели на адекватность

преобразуем массив в датафрейм и посмотрим у что модель предсказала тариф для всех клиентов:

In [89]:
df_predictions = pd.DataFrame(predictions, 
             columns=['predictions_num'])

In [90]:
df_predictions.value_counts()

predictions_num
0                  501
1                  142
dtype: int64

531+112=643. Именно такой размер у тестовой выборки. Значит модель нигде не промахнулась.


<div class="alert alert-warning">
    
<b>Комментарий ревьюера:</b>⚠️Проверка на адекватность, это сравнение с константной моделью. Вот ссылка посмотри пожалуйста. https://russianblogs.com/article/3021544232/, http://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html </div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Внизу проверил на адекватность. Сравнил точность константной модели и нашей модели:</div>

проверим модель на адекватность используя модель DummyClassifier. Эта модель будет всегда предсказывать самое частое значение:

In [91]:
dummy = DummyClassifier(strategy="most_frequent")

In [92]:
dummy.fit(features_train,target_train)

DummyClassifier(strategy='most_frequent')

In [93]:
dummy_predictions=dummy.predict(features_test)
print(dummy_predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

видно что модель всегда пресказывает 0, то есть самое частое значение. Посмотрим на точность константной модели:

In [94]:
accuracy_score(target_test,dummy_predictions)

0.6967340590979783

посмотрим какая получилась точность нашей модели:

In [95]:
accuracy_score(target_test, predictions)

0.8149300155520995

Точность равна 0.82. Точно лучше чем точность константной модели :)

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 4: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
